In [26]:
from openai import AzureOpenAI
import json
import os
import Functions
from Functions import get_weather_for_city, get_restaurants_for_city,get_hotels_for_city
import time

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

# Create a client
client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

In [12]:
# Create a vector store
vector_store = client.beta.vector_stores.create(name="Contoso Hiking Products")

# Ready the files for upload to OpenAI
file_paths = ["./data/contoso/product_info_1.md", 
              "./data/contoso/product_info_2.md",
              "./data/contoso/product_info_3.md",
              "./data/contoso/product_info_4.md",
              "./data/contoso/product_info_5.md",
              "./data/contoso/product_info_6.md",
              "./data/contoso/product_info_7.md",
              "./data/contoso/product_info_8.md",
              "./data/contoso/product_info_9.md",
              "./data/contoso/product_info_10.md",
              "./data/contoso/product_info_11.md",
              "./data/contoso/product_info_12.md",
              "./data/contoso/product_info_13.md",
              "./data/contoso/product_info_14.md",
              "./data/contoso/product_info_15.md",
              "./data/contoso/product_info_16.md",
              "./data/contoso/product_info_17.md",
              "./data/contoso/product_info_18.md",
              "./data/contoso/product_info_19.md",
              "./data/contoso/product_info_20.md",
              ]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [27]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)


completed
FileCounts(cancelled=0, completed=20, failed=0, in_progress=0, total=20)
vs_z6wJ11r74njgDY9zmDLcoSak


In [28]:
# Create assistant
assistant = client.beta.assistants.create(
  name="Contoso Travel Assistant",
  instructions="""
  You are a Travel assistant that provides information. 
   When asked for products, you will provide recommendations using the product_info_XX.md files provided to you. 
   You will not provide recommendations outside of those product files 
  """,
  model=azure_deployment, 
  tools=Functions.tools_function,
  tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
  temperature=1,
  top_p=1
)

# Create thread
thread = client.beta.threads.create()
print(thread)



Thread(id='thread_no7ImqqAVrlOZ1T3WOGfTIkd', created_at=1729063416, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [29]:
prompt_prefix = """
  """

user_question ="""What hiking activities can I do in New York? 
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=prompt_prefix + user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_weather_for_city": get_weather_for_city,
      "get_restaurants_for_city": get_restaurants_for_city,
      "get_hotels_for_city": get_hotels_for_city
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_weather_for_city":
        function_response = get_weather_for_city(city=function_args.get("city"))
        print("retrieved weather info")
      elif function_name == "get_restaurants_for_city":
        function_response = get_restaurants_for_city(city=function_args.get("city"))
        print("retrieved restaurant info")
      elif function_name == "get_hotels_for_city":
        function_response = get_hotels_for_city(city=function_args.get("city"))
        print("retrieved hotel info")    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
  else:
    print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

in_progress
retrieved restaurant info
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
incomplete
Given the weather in New York with a temperature of 8.3°C (or 46.9°F), clear conditions, and nightfall, I'd recommend you consider these products for hiking:

1. Alpine Explorer Tent: This tent has great ventilation and a room divider, making it an excellent choice for hiking. It's easy to set up and spacious enough for comfortable camping【10:0†product_info_8.md】.

2. TrekReady Hiking Boots: With the weather being relatively cold, these boots can provide much-needed warmth and support while hiking. It's important to wear them correctly, keeping them clean, and maintaining them for the best rugged hiking experience【10:1†product_info_4.md】.

Please note that these are just recommendations and you should always consider your comfort, safety, and the specific requirements of your hiking trip.


In [23]:
prompt_prefix = """
  """

user_question ="""Based on the weather in New York, what products can you recommend??
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=prompt_prefix + user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_weather_for_city": get_weather_for_city,
      "get_restaurants_for_city": get_restaurants_for_city,
      "get_hotels_for_city": get_hotels_for_city
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_weather_for_city":
        function_response = get_weather_for_city(city=function_args.get("city"))
        print("retrieved weather info")
      elif function_name == "get_restaurants_for_city":
        function_response = get_restaurants_for_city(city=function_args.get("city"))
        print("retrieved restaurant info")
      elif function_name == "get_hotels_for_city":
        function_response = get_hotels_for_city(city=function_args.get("city"))
        print("retrieved hotel info")    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
  else:
    print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

in_progress
retrieved weather info
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
Given the weather in New York with a temperature of 8.3°C (or 46.9°F), clear conditions, and nightfall, I'd recommend you consider these products for hiking:

1. Alpine Explorer Tent: This tent has great ventilation and a room divider, making it an excellent choice for hiking. It's easy to set up and spacious enough for comfortable camping【10:0†product_info_8.md】.

2. TrekReady Hiking Boots: With the weather being relatively cold, these boots can provide much-needed warmth and support while hiking. It's important to wear them correctly, keeping them clean, and maintaining them for the best rugged hiking experience【10:1†product_info_4.md】.

Please note that these are just recommendations and you should always consider your comfort, safety, and the specific requirements of your hiking trip.
